# Train and export TensorFlow Serving model

In [17]:
def wrapper():

    import os
    import sys
    import uuid
    import random
    import tensorflow as tf
    
    from tensorflow.examples.tutorials.mnist import input_data as mnist_input_data

    from hops import model, hdfs

    training_iteration=random.randint(10, 200)
    
    mnist_dataset_folder = hdfs.copy_to_local("TourData/mnist/MNIST_data")

    # Train model
    print('Training model...')
    mnist = mnist_input_data.read_data_sets(mnist_dataset_folder, one_hot=True)
    sess = tf.InteractiveSession()
    serialized_tf_example = tf.placeholder(tf.string, name='tf_example')
    feature_configs = {'x': tf.FixedLenFeature(shape=[784], dtype=tf.float32),}
    tf_example = tf.parse_example(serialized_tf_example, feature_configs)
    x = tf.identity(tf_example['x'], name='x')  # use tf.identity() to assign name
    y_ = tf.placeholder('float', shape=[None, 10])
    w = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    sess.run(tf.global_variables_initializer())
    y = tf.nn.softmax(tf.matmul(x, w) + b, name='y')
    cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
    values, indices = tf.nn.top_k(y, 10)
    table = tf.contrib.lookup.index_to_string_table_from_tensor(
        tf.constant([str(i) for i in range(10)]))
    prediction_classes = table.lookup(tf.to_int64(indices))
    for _ in range(training_iteration):
        batch = mnist.train.next_batch(16)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float')) 
    acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels})
    print('Training Accuracy: {}'.format(acc))
    print('Done training!')

    # Export model
    # WARNING(break-tutorial-inline-code): The following code snippet is
    # in-lined in tutorials, please update tutorial documents accordingly
    # whenever code changes.

    export_path = os.getcwd() + '/model-' + str(uuid.uuid4())
    print('Exporting trained model to: {}'.format(export_path))
    builder = tf.saved_model.builder.SavedModelBuilder(export_path)

    # Build the signature_def_map.
    classification_inputs = tf.saved_model.utils.build_tensor_info(
        serialized_tf_example)
    classification_outputs_classes = tf.saved_model.utils.build_tensor_info(
        prediction_classes)
    classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

    classification_signature = (
        tf.saved_model.signature_def_utils.build_signature_def(
              inputs={
                  tf.saved_model.signature_constants.CLASSIFY_INPUTS:
                      classification_inputs
              },
              outputs={
                  tf.saved_model.signature_constants.CLASSIFY_OUTPUT_CLASSES:
                      classification_outputs_classes,
                  tf.saved_model.signature_constants.CLASSIFY_OUTPUT_SCORES:
                      classification_outputs_scores
              },
              method_name=tf.saved_model.signature_constants.CLASSIFY_METHOD_NAME))

    tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
    tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

    prediction_signature = (
          tf.saved_model.signature_def_utils.build_signature_def(
              inputs={'images': tensor_info_x},
              outputs={'scores': tensor_info_y},
              method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

    legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
    builder.add_meta_graph_and_variables(
          sess, [tf.saved_model.tag_constants.SERVING],
          signature_def_map={
              'predict_images':
                  prediction_signature,
              tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
                  classification_signature,
          },
          legacy_init_op=legacy_init_op)

    builder.save()

    print('Done exporting!')
    
    metrics = metrics={'accuracy': acc}

    model.export(export_path, "mnist", metrics=metrics)
    
    return metrics

In [18]:
from hops import experiment

experiment.launch(wrapper, name='mnist serving example', metric_key='accuracy')

Finished Experiment 

('hdfs://10.0.2.15:8020/Projects/demo_deep_learning_admin000/Experiments/application_1582818676081_0011_2', {'accuracy': 0.8590999841690063, 'log': 'Experiments/application_1582818676081_0011_2/output.log'})

# Check Model Repository for best model based on accuracy

![Image7-Monitor.png](../../images/models.gif)

# Query Model Repository for best mnist Model

In [19]:
from hops import model
from hops.model import Metric
MODEL_NAME="mnist"
EVALUATION_METRIC="accuracy"

In [20]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

In [21]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: mnist
Model version: 3
{'accuracy': '0.8590999841690063'}

# Create Model Serving of Exported Model

In [22]:
from hops import serving

In [23]:
# Create serving
model_path="/Models/" + best_model['name']
response = serving.create_or_update(model_path, MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model mnist ...
Serving for model mnist successfully created

In [24]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

mnist

In [25]:
# Get serving status
serving.get_status(MODEL_NAME)

'Running'

# Check Model Serving for active servings

![Image7-Monitor.png](../../images/servings.gif)

# Start Model Serving Server

In [26]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

In [27]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [28]:
import numpy as np
TOPIC_NAME = serving.get_kafka_topic(MODEL_NAME)
NUM_FEATURES=784

In [29]:
for i in range(20):
    data = {
                "signature_name": 'predict_images',
                "instances": [np.random.rand(784).tolist()]
            }
    response = serving.make_inference_request(MODEL_NAME, data)
    print(response)

{'predictions': [[0.0375837944, 0.000468574552, 0.610582769, 0.10235218, 0.0104154637, 0.100887753, 0.0218470432, 0.0580656044, 0.0146704987, 0.0431262739]]}
{'predictions': [[0.0224755351, 0.000973382848, 0.445875674, 0.13407363, 0.0100223329, 0.234490916, 0.0110282265, 0.0610676631, 0.0552296042, 0.0247628745]]}
{'predictions': [[0.0312637091, 0.000597591919, 0.592345476, 0.0976754874, 0.0221116357, 0.071305342, 0.0343457684, 0.0988801271, 0.0161056388, 0.0353691205]]}
{'predictions': [[0.062562, 0.000515451888, 0.15809536, 0.120984167, 0.0143883489, 0.547077775, 0.00662078382, 0.0408382304, 0.0181973148, 0.0307206232]]}
{'predictions': [[0.083009623, 0.000603015651, 0.379802734, 0.0894926935, 0.0126961637, 0.236685798, 0.00621740147, 0.109630257, 0.0536771752, 0.0281851478]]}
{'predictions': [[0.0411101133, 0.000423548161, 0.474062294, 0.156157508, 0.00594538124, 0.174169943, 0.0126599204, 0.0548595302, 0.0388185419, 0.0417932607]]}
{'predictions': [[0.0431877114, 0.000698729069, 0.

# Monitor inference requests from Kafka

In [30]:
from hops import kafka
from confluent_kafka import Producer, Consumer, KafkaError
config = kafka.get_kafka_default_config()
config['default.topic.config'] = {'auto.offset.reset': 'earliest'}
consumer = Consumer(config)
topics = [TOPIC_NAME]
consumer.subscribe(topics)

In [31]:
json_schema = kafka.get_schema(TOPIC_NAME)
avro_schema = kafka.convert_json_schema_to_avro(json_schema)

In [32]:
import json
for i in range(0, 10):
    msg = consumer.poll(timeout=1.0)
    if msg is not None:
        value = msg.value()
        try:
            event_dict = kafka.parse_avro_msg(value, avro_schema)
            prediction = json.loads(event_dict["inferenceResponse"])["predictions"][0]
            print("serving: {}, version: {}, timestamp: {},"\
                  "\nrequest: {},\nprediction:{}, http_response_code: {},"\
                  " serving_type: {}\n".format(
                       event_dict["modelName"],
                       event_dict["modelVersion"],
                       event_dict["requestTimestamp"],
                       event_dict["inferenceRequest"],
                       prediction,
                       event_dict["responseHttpCode"],
                       event_dict["servingType"]
            ))
        except Exception as e:
            print("A message was read but there was an error parsing it")
            print(e)
    else:
        print("timeout.. no more messages to read from topic")

timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
timeout.. no more messages to read from topic
serving: mnist, version: 3, timestamp: 1582895681328,
request: {"signature_name": "predict_images", "instances": [[0.8990093619665847, 0.03599006943687033, 0.4415819396481595, 0.04914831316179935, 0.04131369401133822, 0.3045467905968072, 0.42266393805195457, 0.7648496122391041, 0.2180310307589146, 0.10324889912788404, 0.21501864383810987, 0.28574407812989655, 0.5393280563902804, 0.2915672838097664, 0.7355669204407888, 0.35254107344590546, 0.6610614134167231, 0.5526951332723702, 0.17249107926551477, 0.8276271012805713, 0.5491324201583786, 0.40297471978889365, 0.42888788707111114, 0.5751387570529873, 0.4137195699853512, 0.5106107397223072, 0.2510669682268626, 0.9430410575